In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

# Generate Dummy Data
def generate_dummy_data(samples=1000, features=20, classes=2):
    np.random.seed(42)
    X = np.random.rand(samples, features)  # Dummy feature matrix
    y = np.random.randint(0, classes, samples)  # Binary classification labels
    return X, y

# Step 1: Generate dummy breast cancer data
X, y = generate_dummy_data(samples=2000, features=224*224*3, classes=2)
X = X.reshape(-1, 224, 224, 3)  # Reshape for compatibility with transfer learning models
y = tf.keras.utils.to_categorical(y, 2)  # Convert labels to one-hot encoding

# Step 2: Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Define a Transfer Learning Model
def build_transfer_learning_model(base_model_name='VGG16', input_shape=(224, 224, 3), classes=2):
    if base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Unsupported base model. Choose 'VGG16' or 'ResNet50'.")

    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom classification layers
    x = Flatten()(base_model.output)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    return model

# Step 4: Compile and Train the Model
model = build_transfer_learning_model(base_model_name='ResNet50')
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=16)

# Step 5: Evaluate the Model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred, multi_class='ovr'))

# Step 6: Visualization of Training Process
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.legend()
plt.show()

# Step 7: Model Interpretability (Grad-CAM)
def compute_grad_cam(model, img, class_idx):
    grad_model = Model(inputs=model.input, outputs=[model.get_layer("conv5_block3_out").output, model.output])
    with tf.GradientTape() as tape:
        conv_output, predictions = grad_model(img)
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_output)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)

    # Normalize the heatmap
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap.numpy()

# Generate Grad-CAM heatmap for a test sample
sample_img = np.expand_dims(X_test[0], axis=0)
class_idx = np.argmax(y_pred[0])
heatmap = compute_grad_cam(model, sample_img, class_idx)

# Display heatmap
plt.imshow(heatmap, cmap='viridis')
plt.title('Grad-CAM Heatmap')
plt.show()


 9576448/94765736 ━━━━━━━━━━━━━━━━━━━━ 3:42 3us/step